<a href="https://colab.research.google.com/github/LorenzoTinfena/BestSpiderWeb/blob/master/BestSpiderWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BestSpiderWeb
## Problem
A city without roads has a grain producer, an egg producer and a hotel.
The mayor also wants to build a pasta producer and a restaurant in the future. He also wants to build roads like in the picture, so that the producer can easily take the wheat and eggs to make pasta, and the restaurant can easily buy pasta, welcome hotel people, and buy eggs for other preparations.

<img src="https://github.com/LorenzoTinfena/BestSpiderWeb/blob/master/assets/city0.png?raw=1" width="300"/>

**Goal:** to build roads costs, you have to make them as short as possible.

<img src="https://github.com/LorenzoTinfena/BestSpiderWeb/blob/master/assets/city1.png?raw=1" width="400"/>


---


**In other words:** In an Euclidean space there is a graph with constant edges and with 2 types of nodes, one with constant coordinates, the other with a variable coordinates.

**Goal:** To find the positions of the variable nodes in order to have the smaller sum of the length of the edges


# Solution
$$
N_0[c] = \sum_{i \in N}\sum_{v \in P_{N_0  \longleftrightarrow i}}\frac{\sum O_i[c]}{v}
$$
where
*   $$N_0$$

*   $$c$$
coordinates
*   $$N$$
set of nodes with variable coordinates reachable from N with 0 passing only through nodes belonging to N
*   $$O$$
set of nodes with constant coordinates
*   $$O_i$$
set of nodes belonging to "O" adjacent to "i"
*   $$P_{N_0  \rightarrow i}$$
set of all possible paths (infinite for lenght of "N" greater than 1") between node "N with 0" to node "i",
passing only through nodes belonging to N
*   $$v$$
Or path, is a multiplication of the number of edges for all the nodes it crosses, "N with 0" included, "i" included,
(e.g. if it starts from a node that has 7 adjacent edges, then goes through one that has 2,
and ends up with one having 3, the calculation will be 7 * 2 * 3 = 42

# Implementation

In [ ]:
import numpy as np


class Node:
    NoCoordinates = None
    def __init__(self, coordinates: np.ndarray = None):
        self.AdjacentNodes = []
        if coordinates is None:
            self.Constant = False
        else:
            if len(coordinates) != Node.NoCoordinates:
                raise Exception('wrong number of coordinates')
            self.Coordinates = coordinates
            self.Constant = True

    def AddAdjacentNode(self, item: 'Node'):
        self.AdjacentNodes.append(item)

    class VirtualNode:
        def __init__(self, nodeBase: 'Node' = None):
            if nodeBase is not None:
                self.ActualNode = nodeBase
                self.SumConstantNodes = np.zeros(Node.NoCoordinates)
                for item in nodeBase.AdjacentNodes:
                    if item.Constant:
                        self.SumConstantNodes += item.Coordinates
                self.NumTmpPath = len(nodeBase.AdjacentNodes)

        def Copy(self, actualNode: 'Node') -> 'VirtualNode':
            item = Node.VirtualNode()
            item.ActualNode = actualNode
            item.SumConstantNodes = self.SumConstantNodes
            item.NumTmpPath = self.NumTmpPath * len(actualNode.AdjacentNodes)
            return item


def ComputeBestSpiderWeb(variablesNodes: list):
    # initialize coordinates of variables nodes
    for item in variablesNodes:
        item.Coordinates = np.zeros(Node.NoCoordinates)

    # initialize virtual nodes
    virtualNodes = []
    for item in variablesNodes:
        virtualNodes.append(Node.VirtualNode(item))

    # ALGORITHM
    # more iterations means more accuracy (exponential)
    for i in range(40):
        nextVirtualNodes = []
        # iterate through all variables virtual nodes
        for item in virtualNodes:
            # update the coordinates of the actual node
            item.ActualNode.Coordinates += item.SumConstantNodes / item.NumTmpPath
            # iterate through adjacent nodes of the actual node
            for AdjacentItem in item.ActualNode.AdjacentNodes:
                # if the adjacent node is variable add it in a new virtual node (like a tree)
                if not AdjacentItem.Constant:
                    nextVirtualNodes.append(item.Copy(AdjacentItem))
        virtualNodes = nextVirtualNodes

In [ ]:
def main():
    Node.NoCoordinates = 2

    # constant nodes
    Wheat = Node(np.array([0, 0]))
    eggs = Node(np.array([5, 40]))
    hotel = Node(np.array([50, 10]))

    # variables nodes
    pastaProducer = Node()
    restaurant = Node()

    # define edges
    pastaProducer.AddAdjacentNode(Wheat)
    pastaProducer.AddAdjacentNode(eggs)
    pastaProducer.AddAdjacentNode(restaurant)
    restaurant.AddAdjacentNode(pastaProducer)
    restaurant.AddAdjacentNode(eggs)
    restaurant.AddAdjacentNode(hotel)

    ComputeBestSpiderWeb([pastaProducer, restaurant])
    print('pastaProducer: ' + str(pastaProducer.Coordinates))
    print('restaurant: ' + str(restaurant.Coordinates))


if __name__ == '__main__':
    main()

pastaProducer: [ 8.75 21.25]
restaurant: [21.25 23.75]
